# **BERT Based Sentiment Analysis Model Server**
The model used here, was trained with the concept of transfer learning  i.e. taking huggingface transformers pretrained BERT model and further training it on a custom dataset of reviews. this yields a sentiment analysis model based on the prior knowledge of BERT. 
The model server is given a list of texts and outputs a list of labels corresponding to its prediction.
The labels express the sentiment of the writer towards the topic of the text:
0 for negative sentiment, 1 for neutral and 2 for positive.

The model file (~430 MB), can be downloaded to your local environment from: https://iguazio-sample-data.s3.amazonaws.com/models/model.pt

### **Steps**
1. [Setup function parameters](#Setup-function-parameters)
2. [Importing the function](#Importing-the-function)
3. [Testing the function locally](#Testing-the-function-locally)
4. [Testing the function remotely](#Testing-the-function-remotely)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Following packages are required, make sure to install
# !pip install transformers==3.0.2
# !pip install torch

### **Setup function parameters**

In [3]:
# Setting up models path
model_location = 'https://iguazio-sample-data.s3.amazonaws.com/models/model.pt'

### **Importing the function**

In [4]:
import mlrun
mlrun.set_environment(project='function-marketplace')

fn = mlrun.import_function('hub://sentiment_analysis_serving')
fn.apply(mlrun.auto_mount())
fn.add_model('mymodel', model_path=model_location, class_name='SentimentClassifierServing')

# Manually specifying needed packages 
fn.spec.build.commands = ['pip install transformers==3.0.2', 'pip install torch']

# Loading the model takes long times, so extanding timeout so deployment wont fail.
fn.spec.readiness_timeout = 1000 

### **Testing the function locally**

In [5]:
# When mocking, class has to be present
from sentiment_analysis_serving import *

# create an emulator (mock server) from the function configuration)
server = fn.to_mock_server()

> 2021-10-17 06:55:26,964 [info] model mymodel was loaded
> 2021-10-17 06:55:26,969 [info] Initializing endpoint records
> 2021-10-17 06:55:26,991 [info] Loaded ['mymodel']


In [6]:
# KFServing protocol event
event_data = {'inputs' : [# Here we test a pretty straightforward example for positive sentiment.
                          'I had a pleasure to work with such dedicated team. Looking forward to cooperate with each and every one of them again.',
                          'This app is amazingly good.', 
                          # Here we test a pretty straightforward example for bad sentiment.
                          'The new design is awful!',
                          'the mobile app can be really glitchy and is definitely not user friendly',
                          # arguably harder due to misleading words that express, on their own, an opposite sentiment comparing to the full text.
                          'As much as I hate to admit it, the new added feature is surprisingly user friendly.']}

response = server.test("/v2/models/mymodel/predict", body=event_data)
response

{'id': '40270df2843c4afbae53afb94b58664b',
 'model_name': 'mymodel',
 'outputs': [2, 2, 0, 0, 2]}

### **Testing the function remotely**

In [9]:
address = fn.deploy()

> 2021-10-17 07:00:33,444 [info] Starting remote function deploy
2021-10-17 07:00:33  (info) Deploying function
2021-10-17 07:00:33  (info) Building
2021-10-17 07:00:33  (info) Staging files and preparing base images
2021-10-17 07:00:33  (info) Building processor image
2021-10-17 07:00:34  (info) Build complete
2021-10-17 07:02:24  (info) Function deploy complete
> 2021-10-17 07:02:25,851 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-default-sentiment-analysis-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.dev39.lab.iguazeng.com:31010']}


In [13]:
import json
import requests

# using requests to predict
response = requests.put(address + "/v2/models/mymodel/predict", json=json.dumps(event_data))
response.text

'{"id": "40270df2843c4afbae53afb94b58664b", "model_name": "mymodel", "outputs": [2, 2, 0, 0, 2]}'

[Back to the top](#BERT-Based-Sentiment-Analysis-Model-Server)